# Evaluation of a selection of models on 60km -> 2.2km-4x over Birmingham

In [ ]:
%reload_ext autoreload

%autoreload 2

%reload_ext dotenv
%dotenv

import functools
import math
import string

import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from mlde_analysis.data import prep_eval_data
from mlde_analysis import plot_map, SUBREGIONS
from mlde_analysis.ccs import compute_changes, plot_changes, plot_tp_fd, plot_hist_per_tp
from mlde_analysis.distribution import normalized_mean_bias, normalized_std_bias, plot_freq_density, plot_mean_biases, plot_std_biases, rms_mean_bias, rms_std_bias, compute_metrics, DIST_THRESHOLDS
from mlde_analysis.wet_dry import threshold_exceeded_prop_stats, threshold_exceeded_prop_change, plot_threshold_exceedence_errors, THRESHOLDS
from mlde_analysis.display import VAR_RANGES, pretty_table
from mlde_utils import cp_model_rotated_pole, TIME_PERIODS

In [ ]:
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from mlde_analysis.default_params import *

In [ ]:
IPython.display.Markdown(desc)

In [ ]:
EVAL_DS, MODELS = prep_eval_data(sample_configs, dataset_configs, derived_variables_config, eval_vars, split, ensemble_members=ensemble_members, samples_per_run=samples_per_run)
EVAL_DS

In [ ]:
CPM_DAS = { var: EVAL_DS["CPM"][f"target_{var}"] for var in eval_vars }

MODELLABEL2SPEC = { model: {"source": source} | spec for source, models in MODELS.items() for model, spec in models.items() }

In [ ]:
PRED_DAS = { var: xr.concat([ ds[f"pred_{var}"] for ds in EVAL_DS.values() ], dim="model") for var in eval_vars }

In [ ]:
CCS_SEASONS = ["DJF", "JJA"]

In [ ]:
CCS_MODELS = {source: {model: spec for model, spec in mconfigs.items() if spec.get("CCS", False)} for source, mconfigs in MODELS.items() }

CCS_PRED_DAS = {var: PRED_DAS[var].sel(model=list([model for source, models in CCS_MODELS.items() for model in models])) for var in eval_vars }

CCS_DS = xr.combine_by_coords([*list(CPM_DAS.values()), *list(CCS_PRED_DAS.values())])

## Figure: per Time Period per season distribution

* Frequency Density Histogram of rainfall intensities for each time period

In [ ]:
for var in eval_vars:
    fig = plt.figure(layout='constrained', figsize=(3.5*len(CCS_SEASONS), 2*len(TIME_PERIODS)))
    spec = np.array(list([[f"{season} {tp_key}" for season in CCS_SEASONS] for tp_key in TIME_PERIODS.keys() ]))
    
    axd = fig.subplot_mosaic(spec, sharex=True, sharey=True)#, gridspec_kw=dict(height_ratios=[3, 2, 2]), per_subplot_kw={ak: {"projection": cp_model_rotated_pole} for ak in meanb_axes_keys + stddevb_axes_keys})
    for season, seasonal_ccs_ds in CCS_DS.groupby("time.season"):
        if season not in CCS_SEASONS:
            continue
        
        for tp_key, tp_ds in seasonal_ccs_ds.groupby("time_period"):
            ax = axd[f"{season} {tp_key}"]
            hist_data = [ dict(data=model_ds[f"pred_{var}"], label=model, color=MODELLABEL2SPEC[model]["color"]) for model, model_ds in tp_ds.groupby("model") ]
            hrange=VAR_RANGES[var]
            plot_freq_density(hist_data, ax=ax, target_da=tp_ds[f"target_{var}"], legend=(tp_key=="historic" and season=="DJF"), linewidth=1, title=f"{season} {tp_key}", hrange=hrange)
    
            ax.xaxis.label.set_visible(tp_key == "future")    
            
    plt.show()

In [ ]:
for var in eval_vars:
    IPython.display.display_markdown(f"### {var}", raw=True)


    metrics_ds = CCS_DS.groupby("time.season").map(lambda season_ds: season_ds.groupby("time_period").map(lambda tp_ds: compute_metrics(tp_ds[f"pred_{var}"], tp_ds[f"target_{var}"], thresholds=DIST_THRESHOLDS[var])))
        # IPython.display.display_markdown(f"#### {season}", raw=True)
        
    pretty_table(metrics_ds, round=4, dim_order=["season", "time_period", "model", "threshold"])

## Figure: per subdomain per time period per season distribution

In [ ]:
for tp, ds in seasonal_ccs_ds.groupby("time_period"):
    print(tp)

In [ ]:
from collections import deque
for var in eval_vars:
    IPython.display.display_markdown(f"### {var}", raw=True)

    for season, seasonal_ccs_ds in CCS_DS.groupby("time.season"):
        if season not in CCS_SEASONS:
            continue

        fig = plt.figure(figsize=(3.5*len(TIME_PERIODS), 2.5*len(SUBREGIONS)), layout="compressed")

        spec = np.array(list([[f"{srname} {tp_key}" for tp_key in TIME_PERIODS.keys()] for srname in SUBREGIONS.keys()]))
        axd = fig.subplot_mosaic(spec, sharex=True, sharey=True)
        panel_label_iter = iter(string.ascii_lowercase)
        for srname, srdefn in SUBREGIONS.items():
            for tp_key, tp_ds in seasonal_ccs_ds.groupby("time_period"):
            
                tpsrseason_cpm_da = tp_ds[f"target_{var}"].isel(**SUBREGIONS[srname])
                
                srseason_hist_data = [
                    dict(data=model_ds[f"pred_{var}"].isel(**SUBREGIONS[srname]), label=model, color=MODELLABEL2SPEC[model]["color"]) for model, model_ds in tp_ds.groupby("model")
                ]
    
                ax = axd[f"{srname} {tp_key}"]
                if var == "pr":
                    kwargs = {
                        "yscale": "log",
                    }
                else:
                    kwargs = {
                        "yscale": "linear",
                    }
                plot_freq_density(srseason_hist_data, ax=ax, target_da=tpsrseason_cpm_da, linewidth=1, hrange=VAR_RANGES[var], **kwargs)
                ax.set_title(f"{srname} {season} {tp_key}", size="small")

        deque(axd[axlabel.item()].annotate(f"{next(panel_label_iter)}.", xy=(-0.1, 1.0), xycoords=("axes fraction", "axes fraction"), weight='bold', ha="left", va="bottom") for axlabel in np.nditer(spec))
                
        # ax = axd[spec[0][0]]
        # ax.set_xlabel("")
        # ax.get_legend().remove()
        # reg_ax = fig.add_axes([0.8, 0.8, 0.2, 0.2], projection=cp_model_rotated_pole)
        # # reg_ax = fig.subplots(1, subplot_kw = dict(projection=cp_model_rotated_pole))
        # nw_cpm_da = CPM_DAS[var].isel(**SUBREGIONS["NW"])
        # se_cpm_da = CPM_DAS[var].isel(**SUBREGIONS["SE"])
        # plot_map(10*xr.ones_like(nw_cpm_da.isel(ensemble_member=0, time=0)), ax=reg_ax, style=None, cmap="Reds", cl_kwargs=dict(alpha=0.2))
        # plot_map(1*xr.ones_like(se_cpm_da.isel(ensemble_member=0, time=0)), ax=reg_ax, style=None, cmap="Blues", cl_kwargs=dict(alpha=0.2))
        # reg_ax.annotate("NW", xy=(nw_cpm_da.grid_longitude.mean().values.item(), nw_cpm_da.grid_latitude.mean().values.item()), xycoords="data", fontsize="medium", ha="center", va="center")
        # reg_ax.annotate("SE", xy=(se_cpm_da.grid_longitude.mean().values.item(), se_cpm_da.grid_latitude.mean().values.item()), xycoords="data", fontsize="medium", ha="center", va="center")
        # reg_ax.set_extent([-2, 3, -2.5, 2.5], crs=cp_model_rotated_pole)
    
        plt.show()